In [1]:
import pandas as pd
import json
from google.cloud import bigquery
import os
from dotenv import load_dotenv
import requests

In [2]:
# mapbox
token = os.getenv("MAPBOX_TOKEN")

# bigquery
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "credentials.json"
client = bigquery.Client()
project_id = os.getenv("PROJECT_ID")
dataset = os.getenv("DATASET")
table = os.getenv("TABLE")

In [8]:
# get data
query = """
    select * from `{}.{}.{}` where cargo = "Diputados nacionales"
""".format(project_id, dataset, table)

resultado = client.query(query).to_dataframe()

res = requests.get(
    "https://guidocaru.github.io/municipios_geo/municipios_pba_geo.json")
municipios_geo = json.loads(res.text)

# acorta el df de resultados para test
resultado = resultado[resultado['paso'] == 0]

resultado = resultado.groupby(['fecha', 'municipio', 'candidatura', 'id_concat']).agg(
    {'votos': 'sum'}).reset_index()

resultado['votos_perc'] = (resultado['votos'] / resultado.groupby(
    ['fecha', 'municipio', 'id_concat'])['votos'].transform('sum')) * 100

candidaturas = resultado["candidatura"].unique()

print(resultado.to_string())

      fecha                     municipio                                     candidatura id_concat   votos  votos_perc
0      2021                    25 de Mayo                                       + VALORES    020121     485    2.060761
1      2021                    25 de Mayo                                 AVANZA LIBERTAD    020121    1160    4.928829
2      2021                    25 de Mayo  FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD    020121     646    2.744848
3      2021                    25 de Mayo                                 FRENTE DE TODOS    020121    7918    33.64351
4      2021                    25 de Mayo                            FRENTE VAMOS CON VOS    020121     947    4.023794
5      2021                    25 de Mayo                                          JUNTOS    020121   11281   47.932866
6      2021                    25 de Mayo                                         blancos    020121     887    3.768855
7      2021                    25 de May

In [9]:
dff = resultado[resultado['candidatura'] == "FRENTE DE TODOS"]
print(dff.to_string())

      fecha                     municipio      candidatura id_concat   votos  votos_perc
3      2021                    25 de Mayo  FRENTE DE TODOS    020121    7918    33.64351
14     2021                    9 de Julio  FRENTE DE TODOS     02083    7053   23.109436
25     2021                 Adolfo Alsina  FRENTE DE TODOS      0201    2206   21.884921
36     2021        Adolfo Gonzales Chaves  FRENTE DE TODOS     02056    2861   36.019136
47     2021                       Alberti  FRENTE DE TODOS      0202    2639    35.15854
58     2021               Almirante Brown  FRENTE DE TODOS      0203  145039   45.806081
69     2021                     Arrecifes  FRENTE DE TODOS     02010    5760   29.973461
80     2021                    Avellaneda  FRENTE DE TODOS      0204   87156   41.665153
91     2021                      Ayacucho  FRENTE DE TODOS      0205    3791   30.701328
102    2021                          Azul  FRENTE DE TODOS      0206   10462   25.417881
113    2021          

In [11]:
max = dff["votos_perc"].max()
min = dff["votos_perc"].min()
max
min

14.952220348510398